- Please run this after running "word2vec-16dim.ipynb"

In [6]:
from sklearn.cluster import SpectralClustering # 개체간의 거리를 가지고 만든 adjancency matrix 를 활용해 클러스러팅하는 것
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import cudf, itertools
import cuml, cupy
from cuml.neighbors import NearestNeighbors
import muon
import scanpy as sc # Scanpy는 anndata와 공동으로 구축된 단세포 유전자 발현 데이터를 분석하기 위한 확장 가능한 툴킷이다.
print('We will use RAPIDS version',cudf.__version__)

cudf.set_option("default_integer_bitwidth", 32)
cudf.set_option("default_float_bitwidth", 32)

We will use RAPIDS version 22.12.01


In [7]:
w2v_path = '../../input/preprocess/'
output_path = '../../input/preprocess/'

In [10]:
# w2v = pd.read_parquet(w2v_path + 'test_' + 'w2v_output_16dims.parquet')
w2v = pd.read_parquet(w2v_path + 'w2v_output_16dims.parquet') # word2vec 16차원 가져오기.
w2v = w2v.sort_values('aid').reset_index(drop=True) # aid 순으로 정렬하기.

In [ ]:
# %%time
# X_all = sc.AnnData(X = w2v.iloc[:, 1:].values) #  anndata is a Python package for handling annotated data matrices in memory and on disk, positioned between pandas and xarray.
# sc.pp.neighbors(X_all, use_rep = 'X', n_neighbors=64, method='umap') # 관측치의 이웃 그래프를 계산합니다 (manifold)
# sc.tl.leiden(X_all) # 셀을 하위 그룹으로 군집화합니다

In [14]:
%%time
X_all = sc.AnnData(X = w2v.iloc[:,1:].values) #  anndata is a Python package for handling annotated data matrices in memory and on disk, positioned between pandas and xarray.
sc.pp.neighbors(X_all, use_rep = 'X', n_neighbors=64, method='umap') # 관측치의 이웃 그래프를 계산합니다 (manifold)
sc.tl.leiden(X_all) # # 셀을 하위 그룹으로 군집화합니다

CPU times: user 5h 53min 3s, sys: 5min 21s, total: 5h 58min 25s
Wall time: 5h 50min 14s


In [15]:
# aid_df = w2v[['aid']].copy()
# aid_df['cluster'] = list(X_all.obs['leiden'])
# aid_df['cluster'] = aid_df['cluster'].astype(np.int16)
# aid_df.to_parquet(output_path + 'aid_cluster.parquet')

In [16]:
aid_df = w2v[['aid']].copy()
aid_df['cluster'] = list(X_all.obs['leiden']) # X_all.obs['leiden']을 가져와서 cluster에 저장.
aid_df['cluster'] = aid_df['cluster'].astype(np.int16) # type 변경
aid_df.to_parquet(output_path + 'aid_cluster.parquet') # aid_cluster 생성.